1. Extract keypoints


detection model is centernet

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from utils.utils import get_face_points, normalize_wrt_maximum_distance_point, get_torso_points
from utils.img_utils import draw_key_points_pose, draw_axis
from utils.utils_tflite import initialize_interpreter, resize_preserving_ar, inference_interpreter, pose_from_det

%matplotlib inline

In [3]:
import datetime
import json
import sys
from enum import Enum
from pathlib import Path
from exceptions import *
import argparse

import pandas as pd
import tensorflow as tf
from tensorflow import keras

from load_data.dataset_creation import load_data, calculate_stats_in_dataset
#from models.models import *
from models.model_body import * # new
from models.losses import *
from models.metric import *

from sklearn.model_selection import GroupShuffleSplit
import numpy as np

ModuleNotFoundError: No module named 'pose'

In [4]:
img = cv2.imread(r"..\07\07\free_1_ID07\RGB\000464_RGB.png")
im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(im_rgb.shape)

(1080, 1920, 3)


In [5]:
img = im_rgb

In [6]:
detection_model_path = r"../HHP-Net/centernet/centernet_mobilenetv2_fpn_od/model.tflite"
interpreter_od, input_shape_model_od, input_details_od = initialize_interpreter(detection_model_path)

In [7]:
pose_model_path = r"../HHP-Net/posenet/posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"
interpreter_pose, input_shape_interpreter_pose, input_details_pose = initialize_interpreter(pose_model_path)

In [8]:
resized_img, new_old_shape = resize_preserving_ar(img, input_shape_model_od)
print(new_old_shape)

(0, 140, (320, 180))


In [9]:
print(resized_img.shape)

(320, 320, 3)


In [10]:
boxes, classes, scores, num_det = inference_interpreter(interpreter_od, resized_img, input_details_od)


In [11]:
kpt = pose_from_det(resized_img, boxes, classes, scores, interpreter_pose, input_shape_interpreter_pose, input_details_pose, img, new_old_shape, False, 0.3)


In [10]:
kpt

[[[1.25, 0.3125, 1.0],
  [1.261111111111111, 0.296875, 1.0],
  [1.238888888888889, 0.290625, 1.0],
  [1.1333333333333333, 0.36875, 0.0],
  [1.1444444444444444, 0.275, 1.0],
  [1.1388888888888888, 0.378125, 1.0],
  [0.95, 0.30625, 1.0],
  [1.1, 0.41875, 0.0],
  [0.7833333333333333, 0.35, 1.0],
  [1.0722222222222222, 0.4625, 1.0],
  [0.7722222222222223, 0.465625, 1.0],
  [1.0055555555555555, 0.471875, 1.0],
  [0.8444444444444444, 0.459375, 1.0],
  [1.0777777777777777, 0.5625, 0.0],
  [0.7944444444444444, 0.5625, 0.0],
  [0.7777777777777778, 0.46875, 0.0],
  [0.8277777777777777, 0.5625, 0.0]]]

In [11]:
img_res = img.copy()
for kpt_person in kpt:
        for elem in kpt_person:
            elem[0] = elem[0] * img.shape[0]
            elem[1] = elem[1] * img.shape[1]

pose_id_part_centernet = {
    0:  "Nose",
    1:  "Neck",
    2:  "RShoulder",
    3:  "RElbow",
    4:  "RWrist",
    5:  "LShoulder",
    6:  "LElbow",
    7:  "LWrist",
    8:  "MidHip",
    9:  "RHip",
    10: "RKnee",
    11: "RAnkle",
    12: "LHip",
    13: "LKnee",
    14: "LAnkle",
    15: "REye",
    16: "LEye",
    17: "REar",
    18: "LEar",
    19: "LBigToe",
    20: "LSmallToe",
    21: "LHeel",
    22: "RBigToe",
    23: "RSmallToe",
    24: "RHeel",
    25: "Background"
}

 But from the https://github.com/CMU-Perceptual-Computing-Lab/openpose_train/issues/20
 
 {0,  "Nose"},
        {1,  "LEye"},
        {2,  "REye"},
        {3,  "LEar"},
        {4,  "REar"},
        {5,  "LShoulder"},
        {6,  "RShoulder"},
        {7,  "LElbow"},
        {8,  "RElbow"},
        {9,  "LWrist"},
        {10, "RWrist"},
        {11, "LHip"},
        {12, "RHip"},
        {13, "LKnee"},
        {14, "RKnee"},
        {15, "LAnkle"},
        {16, "RAnkle"},
        {17, "UpperNeck"},
        {18, "HeadTop"},
        {19, "LBigToe"},
        {20, "LSmallToe"},
        {21, "LHeel"},
        {22, "RBigToe"},
        {23, "RSmallToe"},
        {24, "RHeel"},

pose_id_part_openpose = {
    0:  "Nose",
    1:  "Neck",
    2:  "RShoulder",
    3:  "RElbow",
    4:  "RWrist",
    5:  "LShoulder",
    6:  "LElbow",
    7:  "LWrist",
    8:  "MidHip",
    9:  "RHip",
    10: "RKnee",
    11: "RAnkle",
    12: "LHip",
    13: "LKnee",
    14: "LAnkle",
    15: "REye",
    16: "LEye",
    17: "REar",
    18: "LEar",
    19: "LBigToe",
    20: "LSmallToe",
    21: "LHeel",
    22: "RBigToe",
    23: "RSmallToe",
    24: "RHeel",
    25: "Background"
}

In [12]:
len(kpt_person)

17

In [13]:
face_kpt = get_face_points(kpt_person, 'centernet')
torso_kpt = get_torso_points(kpt_person, 'centernet')

In [14]:
kpt_person

[[1350.0, 600.0, 1.0],
 [1362.0, 570.0, 1.0],
 [1338.0, 558.0, 1.0],
 [1224.0, 708.0, 0.0],
 [1236.0, 528.0, 1.0],
 [1230.0, 726.0, 1.0],
 [1026.0, 588.0, 1.0],
 [1188.0, 804.0, 0.0],
 [846.0, 672.0, 1.0],
 [1158.0, 888.0, 1.0],
 [834.0, 894.0, 1.0],
 [1086.0, 906.0, 1.0],
 [912.0, 882.0, 1.0],
 [1164.0, 1080.0, 0.0],
 [858.0, 1080.0, 0.0],
 [840.0, 900.0, 0.0],
 [893.9999999999999, 1080.0, 0.0]]

In [15]:
torso_kpt

[1230.0, 726.0, 1.0, 1026.0, 588.0, 1.0, 1086.0, 906.0, 1.0, 912.0, 882.0, 1.0]

In [16]:
mean_x_face = np.mean([face_kpt[i] for i in range(0, 15, 3) if face_kpt[i] != 0.0])
mean_y_face = np.mean([face_kpt[i + 1] for i in range(0, 15, 3) if face_kpt[i + 1] != 0.0])


In [17]:
mean_x_torso = np.mean([torso_kpt[i] for i in range(0, 12, 3) if torso_kpt[i] != 0.0])
mean_y_torso = np.mean([torso_kpt[i + 1] for i in range(0, 12, 3) if torso_kpt[i + 1] != 0.0])


In [18]:
face_kpt_normalized = np.array(normalize_wrt_maximum_distance_point(face_kpt, mean_x_face, mean_y_face))
torso_kpt_normalized = np.array(normalize_wrt_maximum_distance_point(torso_kpt, mean_x_torso, mean_y_torso))
        

In [20]:
torso_kpt_normalized.shape

(12,)

In [21]:
input_kpts_face = tf.cast(np.expand_dims(face_kpt_normalized, 0), tf.float32)
print(input_kpts_face)

input_kpts_torso = tf.cast(np.expand_dims(torso_kpt_normalized, 0), tf.float32)
print(input_kpts_torso)

tf.Tensor(
[[ 0.72727275  0.11111111  1.          0.90909094 -0.35185185  1.
   0.54545456 -0.537037    1.          0.          0.          0.
  -1.         -1.          1.        ]], shape=(1, 15), dtype=float32)
tf.Tensor(
[[ 1.         -0.264       1.         -0.22522523 -1.          1.
   0.13513513  0.696       1.         -0.9099099   0.568       1.        ]], shape=(1, 12), dtype=float32)


Ok, now we have keypoints for input

In [31]:
#Lshoulder, Rshoulder, Lelbow, Relbow, Lwrist, Rwrist, Lhip, Rhip
#Lshoulder, Rshoulder, Lhip, Rhip


In [ ]:
# the center for torso will be intersection of diagonal


In [ ]:
import datetime
import json
import sys
from enum import Enum
from pathlib import Path
from exceptions import *
import argparse

import pandas as pd
import tensorflow as tf
from tensorflow import keras

from load_data.dataset_creation import load_data, calculate_stats_in_dataset
#from models.models import *
from models.model_body import * # new
from models.losses import *
from models.metric import *

from sklearn.model_selection import GroupShuffleSplit
import numpy as np


class PRE_PROCESS_NETWORK(Enum):
    CENTERNET = 'kpts_centernet_super_new'
    OPENPOSE = 'kpts_openpose'
    MEDIAPIPE = 'kpts_mediapipe'
    ALPHAPOSE = 'kpts_alphapose' #new
    # PRE_PROCESS_NETWORK.MEDIAPIPE.name
    # PRE_PROCESS_NETWORK.MEDIAPIPE.value
    # repr(PRE_PROCESS_NETWORK.MEDIAPIPE)


if __name__=='__main__':
    ap = argparse.ArgumentParser()
    ap.add_argument("-e", "--epochs", type=int, default=250, help="epochs to train", required=False)
    ap.add_argument("-a", "--all-dataset", type=bool, default=True, help="This flag allow to load a small portion of the data", required=False)
    ap.add_argument("-s", "--save-model", type=bool, default=True,
                    help="If the trained model would be saved or not", required=False)
    ap.add_argument("-em", "--extraction-model", type=str,
                    help="This flag allow to load_data preprocessed with the required network", required=True)
    ap.add_argument("-ex", "--extended-model", default=False,
                    help="Flag to use 33 kpts from mediapipe", required=False, action="store_true")
    # ap.add_argument("--centernet", help="Chose mediapipe as kpts folder", required=False, action='store_const', const=PRE_PROCESS_NETWORK.CENTERNET)
    # ap.add_argument("--openpose", help="Chose mediapipe as kpts folder", required=False, action='store_const', const=PRE_PROCESS_NETWORK.OPENPOSE)
    # ap.add_argument("--mediapipe", help="Chose mediapipe as kpts folder", required=False, action='store_const', const=PRE_PROCESS_NETWORK.MEDIAPIPE)
    #
    config = ap.parse_args()
    # # to have a validation appply StratifiedShuffleSplit() on test
    #
    # print(f'all dataset = {config.all_dataset}')
    # print(f'model = {config}')




    SERVER = False
    ALL_DATASET = config.all_dataset  # useful for debug
    EPOCHS = config.epochs
    TRIALS = 1
    EXTENDED = False
    SAVE_MODEL = config.save_model
    match (config.extraction_model.lower()):
        case 'centernet':
            KPTS_EXTRACTOR = PRE_PROCESS_NETWORK.CENTERNET
        case 'mediapipe':
            KPTS_EXTRACTOR = PRE_PROCESS_NETWORK.MEDIAPIPE
            EXTENDED = config.extended_model
        case 'openpose':
            KPTS_EXTRACTOR = PRE_PROCESS_NETWORK.OPENPOSE
        case 'alphapose': # new
            KPTS_EXTRACTOR = PRE_PROCESS_NETWORK.ALPHAPOSE #new
        case _:
            raise NotImplementedError('Check spelling of input values')
    # KPTS_EXTRACTOR = PRE_PROCESS_NETWORK.CENTERNET


    print(f'\n SERVER {SERVER}, ALL_DATASET = {ALL_DATASET}, \n EPOCHS = {EPOCHS}, TRIALS = {TRIALS}\n KPTS_EXTRACTOR = {KPTS_EXTRACTOR.name}\n EXTENDED = {EXTENDED}')


    if SERVER:
        kpts_path = Path('/media/DATA/Datasets/BIWI_processed/') / KPTS_EXTRACTOR.value
        print(f'kpts path ={kpts_path}')
        angles_gt = Path('/media/DATA/Datasets/BIWI/db_annotations')
    else:
        kpts_path = Path(r"../07/07_processed") / KPTS_EXTRACTOR.value
        angles_gt = Path('../07/07_annotations')

    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # try:
        #     tf.config.experimental.set_virtual_device_configuration(gpus[0], [
        #         tf.config.experimental.VirtualDeviceConfiguration(memory_limit=000)])
        # except RuntimeError as e:
        #     print(e)

    # create folder for results
    current_time = datetime.datetime.now()
    today = datetime.date.today()
    d1 = '' + today.strftime("%d-%m-%Y") + '_' + str(current_time.hour) + '_' + str(current_time.minute)+'_'+str(KPTS_EXTRACTOR.name)
    folder_results = Path('/Tensorboard/HPE-Net')
    folder_results = folder_results / d1
    if not folder_results.is_dir():
        folder_results.mkdir(parents=True, exist_ok=False)

    try:
        my_data, my_gt, groups = load_data(kpts_path=kpts_path, angles_gt=angles_gt, server=SERVER,
                                           all_data=ALL_DATASET, extended=EXTENDED)
    except IncorrectInputValue:
        print('data ot gt path are incorrect')
        sys.exit(2)
    mean_uncertainty_in_dataset, std_uncertainty_in_dataset = calculate_stats_in_dataset(my_data)
    print(f'\nUNCERTAINTY: mean = {mean_uncertainty_in_dataset},  var = {std_uncertainty_in_dataset}\n')

    log_dir = "/Tensorboard/HPE-Net/logs/fit/" + datetime.datetime.now().strftime(
            "%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    early_stopping_callback = keras.callbacks.EarlyStopping(
                # Stop training when `val_loss` is no longer improving
                monitor="val_loss",
                # "no longer improving" being defined as "no better than 1e-2 less"
                min_delta=0.001, # almost everythinh is an improvemente
                # "no longer improving" being further defined as "for at least 5 epochs"
                patience=5,
                verbose=1,
                restore_best_weights = False, # take the weights 5 epochs after best
        )

    # my_data = np.ones([100,15], dtype=float)
    # my_gt = np.asarray(tf.cast(np.ones([100,2], dtype=float), dtype=tf.float32))
    # my_gt = np.ones([100, 2], dtype=float)
    # groups = np.ones([100], dtype=float)
    # groups[35:70] = 2 * groups[35:70]
    # groups[70:] = 3 * groups[70:]

    # my_model.summary()
    # keras.utils.plot_model(my_model, "/media/DATA/Users/Federico/my_first_model_with_shape_info.png", show_shapes=True)

    my_results = {}
    my_results['yaw_MAE_hpe'] = []
    my_results['pitch_MAE_hpe'] = []
    my_results['roll_MAE_hpe'] = []

    # https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators
    # SS = GroupShuffleSplit(n_splits=5, test_size=0.20, random_state=0)
    # for train_index, test_index in SS.split(my_data, groups=groups):
    i=0
    slicer = GroupShuffleSplit(n_splits=10, random_state=i, train_size=18)

    for train_index, test_index in slicer.split(my_data, my_gt, groups):
        X_train, X_test = my_data[train_index], my_data[test_index]
        y_train, y_test = my_gt[train_index], my_gt[test_index]
        if EXTENDED:
            print('EXTENDED model')
            my_model = hhp_net_body(mean=mean_uncertainty_in_dataset, std=std_uncertainty_in_dataset, alpha=1, only_hands = False) # new it was extended
        else:
            my_model = hhp_net_body(mean=mean_uncertainty_in_dataset, std=std_uncertainty_in_dataset, alpha=1)

        # dataset circa 17'000 examples
        # to start decaying at 50th epochs
        # 50 * (num_samples_of_whole_dataset / batch_size) = 50*(17000/100)= 8500
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=1e-3,
                decay_steps=1000, # new
                decay_rate=0.5,
                staircase=True)

        my_model.compile(
                optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),  # Optimizer
                # Loss function to minimize
                loss=[Mse_loss_single_output_with_uncertainty(), Mse_loss_single_output_with_uncertainty(),
                      Mse_loss_single_output_with_uncertainty()],
                # List of metrics to monitor
                # metrics=[keras.metrics.MeanSquaredError()],
                metrics={'yaw'  : [Mean_Absolute_Error_HPE(), Save_Uncertainty_deg()],
                         'pitch': [Mean_Absolute_Error_HPE(), Save_Uncertainty_deg()],
                         'roll' : [Mean_Absolute_Error_HPE(), Save_Uncertainty_deg()]},
                # metrics = {'yaw': keras.metrics.MeanSquaredError(), 'pitch': keras.metrics.MeanSquaredError(), 'roll': keras.metrics.MeanSquaredError()},
        )

        my_model.summary()
        # keras.utils.plot_model(my_model, "/media/DATA/Users/Federico/my_first_model_with_shape_info.png", show_shapes=True)
        print('\n')
        print(f"TRAINING {i}---------------")

        # print("tarin indexes %s\ntest indexes %s\n" % (train_index, test_index))
        print(f'groups for training{np.unique(groups[train_index])}\ngroups for test {np.unique(groups[test_index])}')

        # print(f'X_train.shape = {X_train.shape}')  # = (num_samples,15)
        # print(f'y_train.shape = {y_train.shape}')  # = (num_samples,2)
        #
        # print(f'X_test.shape = {X_test.shape}')  # = (num_samples,15)
        # print(f'y_test.shape = {y_test.shape}')  # = (num_samples,2)

        # train_dataset = tf.data.Dataset.from_tensor_slices((X_train, (y_train, y_train, y_train)))
        # test_dataset = tf.data.Dataset.from_tensor_slices((X_test, (y_test, y_test, y_test)))
        # {"a": [1, 2], "b": [3, 4]}
        train_dataset = tf.data.Dataset.from_tensor_slices(({'input': X_train, }, {'yaw'  : y_train[:, [0]].squeeze(),
                                                                                   'pitch': y_train[:, [1]].squeeze(),
                                                                                   'roll' : y_train[:, [2]].squeeze()}))
        test_dataset = tf.data.Dataset.from_tensor_slices(({'input': X_test, }, {'yaw'  : y_test[:, [0]].squeeze(),
                                                                                 'pitch': y_test[:, [1]].squeeze(),
                                                                                 'roll' : y_test[:, [2]].squeeze()}))

        train_dataset = train_dataset.shuffle(buffer_size=300, reshuffle_each_iteration=True) # new it was 5000

        # print(f'test_dataset ')
        # print(list(test_dataset.as_numpy_iterator())[:2])

        BATCH_SIZE = 64
        train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
        test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)

        # print(f'test_dataset batchsized ')
        # print(list(test_dataset.as_numpy_iterator())[:2])

        history = my_model.fit(train_dataset, epochs=EPOCHS,
                               callbacks=[tensorboard_callback, early_stopping_callback], validation_data=test_dataset
                               )

        print('\n')
        print(f"EVALUATION {i}---------------")

        # print(f'result: yaw, pitch, roll \n{y_test[0:10]}')
        # print(  my_model(X_test[0:10], training=False))

        results = my_model.evaluate(test_dataset, return_dict=True)

        my_results['yaw_MAE_hpe'].append(results['yaw_MAE_hpe'])
        my_results['pitch_MAE_hpe'].append(results['pitch_MAE_hpe'])
        my_results['roll_MAE_hpe'].append(results['roll_MAE_hpe'])

        hist_df = pd.DataFrame(history.history)

        my_dictionary = {'random state'  : i, 'results': results, 'batch size': BATCH_SIZE,
                         'KPTS_EXTRACTOR': KPTS_EXTRACTOR.name, 'training':str(np.unique(groups[train_index])), 'test':str(np.unique(groups[test_index]))}
        my_result_file = folder_results / f'Training_logs_{i}.json'
        with open(str(my_result_file), 'w') as f:
            json.dump(my_dictionary, f)
        hist_json_file = folder_results / f'history_{i}.csv'
        with open(hist_json_file, mode='w') as f:
            hist_df.to_csv(f)


        if SAVE_MODEL:
            my_model.save(folder_results / f'hpe_net_{KPTS_EXTRACTOR.name}_{i}')

        i=i+1

    print('\n')
    print(
        f'\n SERVER {SERVER}, ALL_DATASET = {ALL_DATASET}, \n EPOCHS = {EPOCHS}, TRIALS = {TRIALS}\n KPTS_EXTRACTOR = {KPTS_EXTRACTOR.name}\n')
    print(f"FINAL REUSLT ---------------")
    print(my_results)

    mae_file = folder_results / f'MAE.csv'
    df_results = pd.DataFrame(my_results)
    with open(mae_file, mode='w') as f:
        df_results.to_csv(f)